In [ ]:
!pip install torch
!pip install datasets
!pip install wandb
!pip install transformers==4.28.0
! pip install -U git+https://github.com/huggingface/accelerate.git

In [ ]:
%env "WANDB_API_KEY" "ac00fe143a161e86bd1a8e56b299cc151e02fa1a"
%env "WANDB_USERNAME" "water7arc"
import wandb
wandb.login(key="ac00fe143a161e86bd1a8e56b299cc151e02fa1a")

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DefaultDataCollator
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer, EarlyStoppingCallback

In [ ]:
from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        context = self.data[index]["context"]
        answer_start = self.data[index]["answers"]["answer_start"][0]
        answer_end = answer_start + len(self.data[index]["answers"]["text"][0])
        new_context = context[:answer_start] + \
                                      "</s>" + context[answer_start:answer_end] + "</s>" + context[answer_end:]

        tokenized_in = self.tokenizer(
            new_context,
            text_target=self.data[index]["question"],
            max_length=384,
            truncation="only_first",
            padding="max_length",
            return_tensors="pt",
        )
        tokenized_in["labels"][tokenized_in["labels"] == self.tokenizer.pad_token_id] = -100
        return {"input_ids": tokenized_in["input_ids"][0], "labels": tokenized_in["labels"][0],
                "attention_mask": tokenized_in["attention_mask"][0]}

    def __len__(self):
        return len(self.data)

In [ ]:
squad = load_dataset("squad")

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

tokenized_squad_train = MyDataset(squad["train"], tokenizer)
tokenized_squad_val = MyDataset(squad["validation"], tokenizer)

data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir="models/t5-base-fine_tune/",
    evaluation_strategy="steps",
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=50,
    push_to_hub=False,
    report_to=["wandb"],
    save_steps=1000,
    save_total_limit=5,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end = True,
    # no_cuda=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad_train,
    eval_dataset=tokenized_squad_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

trainer.train()